In [112]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import re
from collections import Counter

sys.path.append('/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts')
import configuration_util
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import mutation_modeling_util
import get_gene_and_cohort_list_utils
import clonality_analysis_util
import math

filePathDict = configuration_util.get_all_files_path_dict()

In [4]:
phasingData = filePathDict['PHASING_DATA']

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (5,45,92,93,112,113) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
allHypermutantIds = get_gene_and_cohort_list_utils.get_all_hypermutant_ids(
    hypermutantIdDir=filePathDict['HYPERMUTATION_STATUS_IDS'])
phasingHyper = phasingData[phasingData['Tumor_Sample_Barcode'].isin(allHypermutantIds)]

In [45]:
#reload(clonality_analysis_util) #for some reason this isnt working
whitelistFlatGenomeCases = clonality_analysis_util.get_facets_whitelist()
phasingHyper['adjPhase'] = phasingHyper.apply(lambda row: 
    'trans' if row['phase'] == 'trans or separate cells' and row['Tumor_Sample_Barcode'] in whitelistFlatGenomeCases
    else row['phase'], axis=1)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [152]:
#Prepare data for plotting
plotThresh = 5
genesToHighlight = [key for key, value in dict(phasingHyper[(phasingHyper['oncogenic.1'].notnull()) & (phasingHyper['oncogenic.2'].notnull()) & (phasingHyper['adjPhase'].isin(['cis', 'trans']))][
    'Hugo_Symbol'].value_counts()).items() if value > plotThresh]

phasingHyper['label'] = phasingHyper.apply(lambda row: 
                                           '1 or 2 silent' if (row['Variant_Classification.1'] == 'Silent') | (row['Variant_Classification.2'] == 'Silent')
                                           else '1 or 2 VUS'
                                               if (not isinstance(row['oncogenic.1'], str)) | (not isinstance(row['oncogenic.2'], str))
                                           else row['Hugo_Symbol'] if row['Hugo_Symbol'] in genesToHighlight
                                           else 'other_tsg' if row['Role'] == 'TSG'
                                           else 'other_oncogene' if row['Role'] == 'Oncogene'
                                           else 'other', axis=1)

phasingHyper['isTrans'] = phasingHyper['adjPhase'].apply(lambda x: 1 if x == 'trans' else 0 if x == 'cis' else None)
phasingHyperWrite = phasingHyper[phasingHyper['isTrans'].notnull()]


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [154]:
writeDir = os.path.join(os.getcwd(), 'FIGURE3_PLOTTING_FILES')
phasingHyperWrite.to_csv(os.path.join(writeDir, 'figure3ePhasingSummary.tsv'),
                         index=False, sep='\t')